In [86]:
import pandas as pd
import pickle 
from scipy.spatial.distance import cosine
from gensim.models import FastText

import sys
sys.path.append('../code')

from preprocessing import GlowpickPreprocessing

# Load data

In [87]:
reviews = pd.read_csv('../dataset/glowpick_reviews.csv')
products = pd.read_csv('../dataset/glowpick_products.csv')
print('reviews.shape: ',reviews.shape)
print('products.shape: ',products.shape)

reviews.shape:  (5208, 7)
products.shape:  (1183, 8)


# Load saved file

In [42]:
text = pickle.load(open('../saved_file/pre_text.pickle','rb'))
text_word = pickle.load(open('../saved_file/pre_text_word.pickle','rb'))
text_pos = pickle.load(open('../saved_file/pre_text_pos.pickle','rb'))
text_word_pos = pickle.load(open('../saved_file/pre_text_word_pos.pickle','rb'))

In [43]:
vec = pickle.load(open('../saved_file/pre_embed.pickle','rb'))
vec_word = pickle.load(open('../saved_file/pre_embed_word.pickle','rb'))
vec_pos = pickle.load(open('../saved_file/pre_embed_pos.pickle','rb'))
vec_word_pos = pickle.load(open('../saved_file/pre_embed_word_pos.pickle','rb'))

In [44]:
model = FastText.load('../saved_file/model.bin')
model_word = FastText.load('../saved_file/model_word.bin')
model_pos = FastText.load('../saved_file/model_pos.bin')
model_word_pos = FastText.load('../saved_file/model_word_pos.bin')

# Test

In [45]:
wordpath = '../saved_file/replace_word.json'
pospath = '../saved_file/pos_dict.json'

In [58]:
search = ['아버지 생신선물']

In [59]:
GP = GlowpickPreprocessing()
s_text = GP.fit(search)
s_text_word = GP.fit(search, wordfix_path=wordpath)
s_text_pos = GP.fit(search, posfix_path=pospath)
s_text_word_pos = GP.fit(search, wordfix_path=wordpath, posfix_path=pospath)

s_vec = GP.sent2vec(s_text, model)
s_vec_word = GP.sent2vec(s_text_word, model_word)
s_vec_pos = GP.sent2vec(s_text_pos, model_pos)
s_vec_word_pos = GP.sent2vec(s_text_word_pos, model_word_pos)

[Log]: complete stopword
Loading JIT Compiled ChatSpace Model
[Log]: complete spacefix
[Log]: complete stopword
Loading JIT Compiled ChatSpace Model
[Log]: complete spacefix
[Log]: complete wordfix
[Log]: complete stopword
Loading JIT Compiled ChatSpace Model
[Log]: complete spacefix
[Log]: complete posfix
[Log]: complete stopword
Loading JIT Compiled ChatSpace Model
[Log]: complete spacefix
[Log]: complete wordfix
[Log]: complete posfix


# distance

In [60]:
def cosine_dist(m, v):
    dist_arr = np.zeros((m.shape[0]))
    for i, sent in enumerate(m):
        dist = cosine(sent, v[0])
        dist_arr[i] = dist
    return dist_arr

In [61]:
dist = cosine_dist(vec, s_vec)
dist_word = cosine_dist(vec_word, s_vec_word)
dist_pos = cosine_dist(vec_pos, s_vec_pos)
dist_word_pos = cosine_dist(vec_word_pos, s_vec_word_pos)

In [62]:
reviews['dist'] = dist
reviews['dist_word'] = dist_word
reviews['dist_pos'] = dist_pos
reviews['dist_word_pos'] = dist_word_pos

In [ ]:
products.vol_price

# Filtering

In [63]:
cat_lst = products.category.unique()

In [83]:
cat_lst

array(['스킨', '로션', '에센스', '크림', '클렌징', '스크럽/필링', '쉐이빙', '애프터쉐이브', '마스크/팩',
       '메이크업', '선케어', '헤어/바디', '헤어스타일링'], dtype=object)

In [64]:
reviews_prod = pd.merge(reviews, products, on='product_url', how='left')

rate_dict = {
    'best':0,
    'good':1,
    'soso':2,
    'bad':3,
    'worst':4
}
reviews_prod.rate_x = reviews_prod.rate_x.map(rate_dict)

In [127]:
s_dict = {
    #메이크업항목
    'makeup_dict':{
    '비비':['BB','파운데이션','듀얼 스틱','원 플루이드 그린','저스트 스킨','모이스처라이저 B'],
    '톤업':['톤 업','톤 매너','아프리카 버드 옴므 올인원 브라이트닝 에센스'],
    '씨씨':['CC','CC크림','리얼컴플렉션 젤 피니셔','포맨 히든스틱'],
    '쿠션':['쿠션크림'],
    '립':['밤','듀얼 스틱'],
    '브로우':['눈썹','마스터 커버스틱','숯 유어셀프'],
    },

    #헤어/바디 항목
    'hair_body_dict':{
    '바디워시':['워시','바디 워시','비어 원샷 클렌저 포맨','맨즈 시트러스 민트 3 IN 1'],
    '샴푸':['방 덴시피크 옴므','두쉐르 베제딸 뿌르옴므','또니끄 비비휘앙 뿌르옴므','뱅 떼에스 뿌르옴므','뱅 트레땅 알라 프로폴리스 뿌르옴므'],
    '샤워젤':['샤워','샤워 젤'],
    '스프레이':['퍼퓸건','미스트','라스트 샷'],
    '로션':['크림','핸드크림','핸드','맨올로지 101 에너자이징 바디 에멀젼'],
    '데오드란트':['데오도란트','맨 블랙 앤 화이트 롤 온','매너 키트','데오도런트'],
    '오일':['맨 퓨어-포먼스 컴포지션']
    },

    #헤어스타일링 항목 
    'hairstyle_dict':{
    '왁스':['무빙러버','젤','하드','헤어잼','크래프트 클레이 리모델러블 매트 텍스처라이저','알앤비','포맨 오리지널 슈퍼 매트','버번 바닐라 & 텐저린 헤어 텍스처라이저','맨 퓨어-포먼스 그루밍 클레이'],
    '컬':['볼륨미아 볼륨 무스']
    }
}

In [115]:
s = ['가성비','CC']

In [135]:
for cat, dic in s_dict.items():
    for k, v in dic.items():
        if k in s:
            inter_word = v
            continue
        for i in v:
            if i in s:
                inter_word = v

In [136]:
print(inter_word)

['CC', 'CC크림', '리얼컴플렉션 젤 피니셔', '포맨 히든스틱']


In [137]:
features = ['product_url','brand','nb_reviews','sex','age_skin_type','rate_x','content','product']

In [138]:
inter_idx = []
for i, p in enumerate(temp['product']):
    for c in inter_word:
        if c in p:
            inter_idx.append(i)

f = set(s_text[0]) & set(cat_lst)
if len(f) > 0:
    temp = reviews_prod[['dist'] + features]
    temp = temp[temp.category.isin(f)].sort_values(by=['rate_x','dist']).drop_duplicates()
else:
    temp = reviews_prod[['dist'] + features].sort_values(by=['rate_x','dist']).drop_duplicates()
temp.head()

,dist,product_url,brand,nb_reviews,sex,age_skin_type,rate_x,content,product
2352,0.000180,/product/93896,미샤,(34),f,31세 · 복합성 ·,0,평가단 평이 다 좋지만은 않네요 저는 평가단은 아니구.. 이거 아빠드렸는데 엄청 좋...,포맨 리프레시 올인원 트리트먼트 [톤업에센스]
4505,0.000182,/product/34094,아이오페,(103),f,29세 · 건성 ·,0,남자친구에게발렌타인선물로 줬는데 진짜 남자용이라그런지 남자피부톤에 딱맞고 한번만해도...,맨 에어쿠션® [SPF50+/PA+++]
2130,0.000205,/product/6892,이니스프리(innisfree),(238),f,27세 · 민감성 ·,0,남친에게 선물 이후로 이것만 쓰더라,포레스트 포맨 모이스처 올인원 에센스
1171,0.000209,/product/64025,이니스프리(innisfree),(6),f,21세 · 건성 ·,0,아빠 생신선물로 같이 사드린건데 후기 남기고 싶어서 제품 등록 신청했어요 ... ♥...,포레스트 포맨 프리미엄 스킨
58,0.000218,/product/7212,이니스프리(innisfree),(81),f,22세 · 복합성 ·,0,남자친구선물해줫는데 매우잘쓴다 그래서만족중,포레스트 포맨 모이스처 스킨


In [74]:
f = set(s_text[0]) & set(cat_lst)
if len(f) > 0:
    temp = reviews_prod[['dist_word'] + features]
    temp = temp[temp.category.isin(f)].sort_values(by=['rate_x','dist_word']).head(20)
else:
    temp = reviews_prod[['dist_word'] + features].sort_values(by=['rate_x','dist_word']).head(20)
temp

,dist_word,product_url,brand,nb_reviews,sales_rank,sex,age_skin_type,rate_x,content,product
3027,0.000491,/product/3689,우르오스,(53),1,f,17세 · 복합성 ·,0,아빠 선물로 드렸는데 좋아하시네요!짱짱,스킨워시
80,0.000524,/product/7212,이니스프리(innisfree),(81),1,f,27세 · 건성 ·,0,남자친구 선물로 줬는데 되게 만족스럽다고 한다~,포레스트 포맨 모이스처 스킨
3029,0.000954,/product/3689,우르오스,(53),1,f,2018세 · 복합성 ·,0,오빠 선물로 줬는데 오빠가 짱좋데용😘,스킨워시
3038,0.001626,/product/3689,우르오스,(53),1,f,20세 · 중성 ·,0,남친 줬더니 행보케하더랔ㅋㅋㅋ,스킨워시
92,0.003487,/product/4108,비오템옴므,(65),2,f,17세 · 지성 ·,0,아빠 생신 선물로 보습을 위해 사드렸는데 정말 좋아하셨어요!,아쿠아파워 토너
970,0.003650,/product/8527,헤라 (HERA),(15),22,f,25세 · 복합성 ·,0,남자친구 선물로 선물로 사줬는데 예민한 피부인데도 좋다고하네요 특히 냄새가 마음에 ...,옴므 블랙 퍼펙트 스킨
3591,0.003971,/product/79342,러쉬(LUSH),(10),4,f,27세 · 민감성 ·,0,남자친구 선물해줬는데 진짜 사용감이 좋다고 하더라구요!,프린스
2453,0.003994,/product/5697,미샤,(14),18,f,20세 · 복합성 ·,0,저희 아빠 이거 쓰시던데 편하대요!,포맨 더 퍼스트 트리트먼트 에센스
3153,0.004099,/product/3459,랩시리즈,(21),4,f,25세 · 건성 ·,0,기념일 선물로 남친 사줬는데 되게 좋다네요~ 이거없음 씻질못함! 나중에 다떨어지면 ...,멀티-액션 훼이스 워시
2279,0.004102,/product/11671,랩시리즈,(22),9,f,25세 · 건성 ·,0,남친한테 기념일 선물로 사줬는데 좋아하네요~ 역시 남자는 랩시리즈인가봄!,프로 엘에스 올인원 훼이스 트리트먼트


In [75]:
f = set(s_text[0]) & set(cat_lst)
if len(f) > 0:
    temp = reviews_prod[['dist_pos'] + features]
    temp = temp[temp.category.isin(f)].sort_values(by=['rate_x','dist_pos']).head(20)
else:
    temp = reviews_prod[['dist_pos'] + features].sort_values(by=['rate_x','dist_pos']).head(20)
temp

,dist_pos,product_url,brand,nb_reviews,sales_rank,sex,age_skin_type,rate_x,content,product
3512,0.000083,/product/2324,니베아 맨,(117),1,f,19세 · 건성 ·,0,아빠에게 생신선물 해드럈는데 어디서 샀냐고 계속 물어보시는 제품 민감하지 않아서 좋으단,센스티브 쉐이빙 폼
1271,0.000092,/product/7200,이니스프리(innisfree),(58),1,f,19세 · 복합성 ·,0,아빠한테 선물해드렸는데 잘 쓰시고 계세요! 뭐 안나고 끈적이지 않아용 무난하게 쓰기...,포레스트 포맨 모이스처 로션
782,0.000093,/product/74074,캘빈클라인,(16),11,f,31세 · 복합성 ·,0,아빠선물로 드렸고 좋아허심ㅠ 2년째 쓰시는제품~/순해요,원 스킨 모이스처라이저
2708,0.000105,/product/78519,불독,(48),2,f,30세 · 민감성 ·,0,남자친구가 젤 좋아하는 화장품이예요 그래서 선물로 사주기 좋아요 이번에 올리브영에서...,오일 컨트롤 수분크림
1171,0.000149,/product/64025,이니스프리(innisfree),(6),58,f,21세 · 건성 ·,0,아빠 생신선물로 같이 사드린건데 후기 남기고 싶어서 제품 등록 신청했어요 ... ♥...,포레스트 포맨 프리미엄 스킨
3027,0.000156,/product/3689,우르오스,(53),1,f,17세 · 복합성 ·,0,아빠 선물로 드렸는데 좋아하시네요!짱짱,스킨워시
2130,0.000157,/product/6892,이니스프리(innisfree),(238),1,f,27세 · 민감성 ·,0,남친에게 선물 이후로 이것만 쓰더라,포레스트 포맨 모이스처 올인원 에센스
3153,0.000169,/product/3459,랩시리즈,(21),4,f,25세 · 건성 ·,0,기념일 선물로 남친 사줬는데 되게 좋다네요~ 이거없음 씻질못함! 나중에 다떨어지면 ...,멀티-액션 훼이스 워시
2279,0.000172,/product/11671,랩시리즈,(22),9,f,25세 · 건성 ·,0,남친한테 기념일 선물로 사줬는데 좋아하네요~ 역시 남자는 랩시리즈인가봄!,프로 엘에스 올인원 훼이스 트리트먼트
58,0.000173,/product/7212,이니스프리(innisfree),(81),1,f,22세 · 복합성 ·,0,남자친구선물해줫는데 매우잘쓴다 그래서만족중,포레스트 포맨 모이스처 스킨


In [76]:
f = set(s_text[0]) & set(cat_lst)
if len(f) > 0:
    temp = reviews_prod[['dist_word_pos'] + features]
    temp = temp[temp.category.isin(f)].sort_values(by=['rate_x','dist_word_pos']).head(20)
else:
    temp = reviews_prod[['dist_word_pos'] + features].sort_values(by=['rate_x','dist_word_pos']).head(20)
temp

,dist_word_pos,product_url,brand,nb_reviews,sales_rank,sex,age_skin_type,rate_x,content,product
1478,0.000040,/product/3691,우르오스,(246),3,f,22세 · 복합성 ·,0,아빠선물해 드렸는데 잘 쓰시더라구요!!,스킨밀크
3027,0.000184,/product/3689,우르오스,(53),1,f,17세 · 복합성 ·,0,아빠 선물로 드렸는데 좋아하시네요!짱짱,스킨워시
2131,0.000326,/product/6892,이니스프리(innisfree),(238),1,f,28세 · 복합성 ·,0,아빠랑 남친이 좋아해요,포레스트 포맨 모이스처 올인원 에센스
813,0.000402,/product/7994,네이처리퍼블릭,(12),12,f,21세 · 민감성 ·,0,아버지가 이벗만3통쓰고계심,아프리카 버드 옴므 올인원 모이스처라이저
814,0.000402,/product/7994,네이처리퍼블릭,(12),15,f,21세 · 민감성 ·,0,아버지가 이벗만3통쓰고계심,아프리카 버드 옴므 올인원 모이스처라이저
815,0.000402,/product/7994,네이처리퍼블릭,(12),11,f,21세 · 민감성 ·,0,아버지가 이벗만3통쓰고계심,아프리카 버드 옴므 올인원 모이스처라이저
924,0.000421,/product/8528,헤라 (HERA),(13),18,f,19세 · 지성 ·,0,"아버지가 쓰시는 제품인ㄷㅔ 향이 진짜 좋아요,, 진짜진짜",옴므 블랙 퍼펙트 플루이드
925,0.000421,/product/8528,헤라 (HERA),(13),20,f,19세 · 지성 ·,0,"아버지가 쓰시는 제품인ㄷㅔ 향이 진짜 좋아요,, 진짜진짜",옴므 블랙 퍼펙트 플루이드
5118,0.000421,/product/33918,카밀,(15),36,f,15세 · 민감성 ·,0,아빠가 쓰시는데 좋으시데욤,카밀핸드 맨
28,0.000434,/product/7212,이니스프리(innisfree),(81),1,f,21세 · 복합성 ·,0,아빠 선물러 사드렸는데 정말 괜찮타고 했던제퓸입니다 스킨 찾으시면 이걸루 사시는게 ...,포레스트 포맨 모이스처 스킨
